# Reglas de asociación- Búsqueda de Patrones
* Preparación: num-> cat (Discretización) (ya que este método es sólo para categorías)
* Aprendizaje: apriori
* Evaluación: soporte y confianza
* Interpretación

In [1]:
#Importamos librerías básicas
import os
import pandas as pd # manipulacion dataframes
import numpy as np  # matrices y vectores
import matplotlib.pyplot as plt #gráfica

renames = {
    "TP9_3_USO": "Uds_No_Residenciales",
    "TP9_2_1_MI": "Uds_Mixtas_Industria",
    "TP9_2_3_MI": "Uds_Mixtas_Servicios",
    "TP9_2_9_MI": "Uds_Mixtas_Otros",
    "TP9_3_1_NO": "Uds_NoResidenciales_Industria",
    "TP9_3_2_NO": "Uds_NoResidenciales_Comercio",
    "TP9_3_3_NO": "Uds_NoResidenciales_Servicios",
    "TP9_3_5_NO": "Uds_NoResidenciales_Institucional",
    "TP9_3_6_NO": "Uds_NoResidenciales_Lote",
    "TP9_3_7_NO": "Uds_NoResidenciales_Parque_ZonaVerde",
    "TP9_3_10_N": "Uds_NoResidenciales_Construccion",
    "TP9_3_99_N": "Uds_NoResidenciales_Otros",
    "TP14_1_TIP": "Viviendas_Tipo_Casa",
    "TP14_3_TIP": "Viviendas_Tipo_Cuarto",
    "TP14_5_TIP": "Viviendas_Tipo_TradicionalEtnica",
    "TP15_3_OCU": "Viviendas_Tipo_Temporal",
    "TP19_EE_2": "Viviendas_SinEnergia",
    "TP19_EE_E1": "Viviendas_Estrato1",
    "TP19_EE_E4": "Viviendas_Estrato4",
    "TP19_EE_E9": "Viviendas_SinEstrato",
    "TP19_ACU_2": "Viviendas_SinAcueducto",
    "TP19_ALC_2": "Viviendas_SinAlcantarillado",
    "TP19_GAS_2": "Viviendas_SinGas",
    "PERSONAS_S": "Pnas_Hogares_Particulares",
    "NMB_LC_CM": "Nombre_Localidad_Comuna",
    "permanencia": "permanencia",
    "persons_TP51_13_ED": "Pnas_Sin_Edu_Formal",
    "persons_TP51SUPERI": "Pnas_Edu_Posgrado",
    "persons_TP51SECUND": "Pnas_Edu_Secundaria",
    "persons_TP51PRIMAR": "Pnas_Edu_Primaria",
    "persons_TP51_99_ED": "Pnas_Edu_SinInformacion",
    "persons_TP34_6_EDA": "Pnas_50_59años",
    "persons_TP34_8_EDA": "Pnas_70_79años",
    "persons_TP34_7_EDA": "Pnas_60_69años",
    "persons_TP34_3_EDA": "Pnas_20_29años",
    "persons_TP34_5_EDA": "Pnas_40_49años",
    "persons_TP34_9_EDA": "Pnas_Mas80años",
    "persons_TP34_4_EDA": "Pnas_30_39años",
    "persons_TP34_2_EDA": "Pnas_10_19años",
    "persons_TP34_1_EDA": "Pnas_0_9años",
    "persons_TP32_1_SEX": "Pnas_Sexo_Hombres",
    "persons_TP32_2_SEX": "Pnas_Sexo_Mujeres",
    "persons_TP51POSTGR": "Pnas_Edu_Posgrado",
    "houses_TP9_1_USO": "Uds_Uso_Residencial",
    "houses_TP19_INTE1": "Viviendas_Con_Internet",
    "houses_TP19_GAS_1": "Viviendas_Con_Gas",
    "houses_TP19_ACU_1": "Viviendas_Con_Acueducto",
    "houses_TP19_GAS_9": "Viviendas_SinInformacion_Gas",
    "houses_TP19_EE_E2": "Viviendas_Estrato2",
    "houses_TP19_EE_E3": "Viviendas_Estrato3",
    "houses_TP19_EE_E5": "Viviendas_Estrato5",
    "houses_TP19_EE_E6": "Viviendas_Estrato6",
    "houses_TP15_1_OCU": "Viviendas_Ocupadas_Permanentemente",
    "houses_TP14_2_TIP": "Viviendas_Tipo_Apartamento",
    "houses_TP9_2_USO": "Uds_Uso_Mixto",
    "houses_TP14_6_TIP": "Viviendas_Tipo_Otros",
    "houses_TP15_2_OCU": "Viviendas_Ocupadas_PnasAusentes",
    "houses_TP14_4_TIP": "Viviendas_Tipo_Tradicional_Indigena",
    "homes_TP27_PERSO": "Numero_Personas"
}

from src.commons.tools import output_path

# 1. Preparación de datos

In [2]:
#Se revisan los tipos de datos
path = os.path.join(output_path, 'descriptive_mining', 'final1_description_categorical.csv')
data=pd.read_csv(path)
data.rename(columns=renames, inplace=True)
data = data.astype({col: str for col in data.columns})
for col in data.columns:
    data[col] = col+' - '+data[col]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 69 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   salario_mes                           252 non-null    object
 1   anios                                 252 non-null    object
 2   DENSIDAD                              252 non-null    object
 3   Uds_No_Residenciales                  252 non-null    object
 4   Uds_Mixtas_Industria                  252 non-null    object
 5   Uds_Mixtas_Servicios                  252 non-null    object
 6   Uds_Mixtas_Otros                      252 non-null    object
 7   Uds_NoResidenciales_Industria         252 non-null    object
 8   Uds_NoResidenciales_Comercio          252 non-null    object
 9   Uds_NoResidenciales_Servicios         252 non-null    object
 10  Uds_NoResidenciales_Institucional     252 non-null    object
 11  Uds_NoResidenciales_Lote        

In [3]:
#Corregir tipos de datos
data = data.astype({col: 'category' for col in data.columns})
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 69 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   salario_mes                           252 non-null    category
 1   anios                                 252 non-null    category
 2   DENSIDAD                              252 non-null    category
 3   Uds_No_Residenciales                  252 non-null    category
 4   Uds_Mixtas_Industria                  252 non-null    category
 5   Uds_Mixtas_Servicios                  252 non-null    category
 6   Uds_Mixtas_Otros                      252 non-null    category
 7   Uds_NoResidenciales_Industria         252 non-null    category
 8   Uds_NoResidenciales_Comercio          252 non-null    category
 9   Uds_NoResidenciales_Servicios         252 non-null    category
 10  Uds_NoResidenciales_Institucional     252 non-null    category
 11  Uds_No

In [4]:
data.head()

,salario_mes,anios,DENSIDAD,Uds_No_Residenciales,Uds_Mixtas_Industria,Uds_Mixtas_Servicios,Uds_Mixtas_Otros,Uds_NoResidenciales_Industria,Uds_NoResidenciales_Comercio,Uds_NoResidenciales_Servicios,...,Viviendas_Tipo_Tradicional_Indigena,Numero_Personas,Desc_Cargo,Proyecto,genero,id_tipo_contrato,id_estado_civil,id_turno,Nombre_Localidad_Comuna,causa_retiro
0,"salario_mes - (1423500.0, 2228170.0]","anios - (27.0, 32.0]","DENSIDAD - (0.0558, 0.15]","Uds_No_Residenciales - (-0.001, 1.0]","Uds_Mixtas_Industria - (-0.001, 5.0]","Uds_Mixtas_Servicios - (-0.001, 1.0]","Uds_Mixtas_Otros - (-0.001, 1.0]","Uds_NoResidenciales_Industria - (-0.001, 0.5]","Uds_NoResidenciales_Comercio - (-0.001, 3.0]","Uds_NoResidenciales_Servicios - (-0.001, 3.0]",...,"Viviendas_Tipo_Tradicional_Indigena - (-0.001,...","Numero_Personas - (2.792, 3.121]",Desc_Cargo - OFICIAL I,Proyecto - CONTRATO SUBESTRUCTURA S2,genero - M,id_tipo_contrato - 5.0,id_estado_civil - 2.0,id_turno - 3.0,Nombre_Localidad_Comuna - LA DESPENSA,causa_retiro - TERMINACION DE CONTRATO
1,"salario_mes - (1299999.999, 1423500.0]","anios - (17.999, 27.0]","DENSIDAD - (0.00656, 0.0276]","Uds_No_Residenciales - (4.0, 10.0]","Uds_Mixtas_Industria - (-0.001, 5.0]","Uds_Mixtas_Servicios - (-0.001, 1.0]","Uds_Mixtas_Otros - (-0.001, 1.0]","Uds_NoResidenciales_Industria - (0.5, 4.0]","Uds_NoResidenciales_Comercio - (-0.001, 3.0]","Uds_NoResidenciales_Servicios - (3.0, 21.0]",...,"Viviendas_Tipo_Tradicional_Indigena - (-0.001,...","Numero_Personas - (2.597, 2.792]",Desc_Cargo - AYUDANTE DE OBRA tasa 6.96,Proyecto - CONTRATO SUBESTRUCTURA S2,genero - M,id_tipo_contrato - 5.0,id_estado_civil - 2.0,id_turno - 3.0,Nombre_Localidad_Comuna - PUENTE ARANDA,causa_retiro - RENUNCIA VOLUNTARIA
2,"salario_mes - (1423500.0, 2228170.0]","anios - (32.0, 39.0]","DENSIDAD - (-0.001, 0.00656]","Uds_No_Residenciales - (10.0, 59.0]","Uds_Mixtas_Industria - (-0.001, 5.0]","Uds_Mixtas_Servicios - (-0.001, 1.0]","Uds_Mixtas_Otros - (-0.001, 1.0]","Uds_NoResidenciales_Industria - (4.0, 18.0]","Uds_NoResidenciales_Comercio - (3.0, 42.0]","Uds_NoResidenciales_Servicios - (3.0, 21.0]",...,"Viviendas_Tipo_Tradicional_Indigena - (-0.001,...","Numero_Personas - (1.499, 2.597]",Desc_Cargo - OFICIAL I,Proyecto - CONTRATO SUBESTRUCTURA S2,genero - M,id_tipo_contrato - 5.0,id_estado_civil - 5.0,id_turno - 3.0,Nombre_Localidad_Comuna - PUENTE ARANDA,causa_retiro - TERMINACION DE CONTRATO
3,"salario_mes - (1423500.0, 2228170.0]","anios - (32.0, 39.0]","DENSIDAD - (-0.001, 0.00656]","Uds_No_Residenciales - (10.0, 59.0]","Uds_Mixtas_Industria - (-0.001, 5.0]","Uds_Mixtas_Servicios - (-0.001, 1.0]","Uds_Mixtas_Otros - (-0.001, 1.0]","Uds_NoResidenciales_Industria - (4.0, 18.0]","Uds_NoResidenciales_Comercio - (3.0, 42.0]","Uds_NoResidenciales_Servicios - (3.0, 21.0]",...,"Viviendas_Tipo_Tradicional_Indigena - (-0.001,...","Numero_Personas - (1.499, 2.597]",Desc_Cargo - OFICIAL I,Proyecto - CONTRATO SUBESTRUCTURA S2,genero - M,id_tipo_contrato - 5.0,id_estado_civil - 5.0,id_turno - 3.0,Nombre_Localidad_Comuna - PUENTE ARANDA,causa_retiro - TERMINACION DE CONTRATO
4,"salario_mes - (1299999.999, 1423500.0]","anios - (17.999, 27.0]","DENSIDAD - (0.0276, 0.0558]","Uds_No_Residenciales - (10.0, 59.0]","Uds_Mixtas_Industria - (-0.001, 5.0]","Uds_Mixtas_Servicios - (1.0, 11.0]","Uds_Mixtas_Otros - (-0.001, 1.0]","Uds_NoResidenciales_Industria - (-0.001, 0.5]","Uds_NoResidenciales_Comercio - (3.0, 42.0]","Uds_NoResidenciales_Servicios - (-0.001, 3.0]",...,"Viviendas_Tipo_Tradicional_Indigena - (-0.001,...","Numero_Personas - (3.121, 4.4]",Desc_Cargo - AYUDANTE DE OBRA tasa 6.96,Proyecto - CONTRATO SUBESTRUCTURA S2,genero - M,id_tipo_contrato - 5.0,id_estado_civil - 2.0,id_turno - 3.0,Nombre_Localidad_Comuna - COMPARTIR,causa_retiro - TERMINACION DE CONTRATO


In [ ]:
#Perfilado de datos
from ydata_profiling import ProfileReport

profile_data=ProfileReport(data, minimal=True) # minimal=False
profile_data

ModuleNotFoundError: No module named 'ydata_profiling'

#Aprendizaje

In [ ]:
from apyori import apriori
#precedente->consecuente
#Soporte: frecuencia(precedente,Consecuente)/total_datos
#Confianza: frecuencia(precedente,Consecuente)/datos_precedente
reglas=apriori(data.values, min_support=0.50, min_confidence=0.8)
reglas=list(reglas)
reglas

# 3. Evaluación

In [ ]:
reglas = pd.DataFrame(reglas)
reglas

In [ ]:
rules = os.path.join(output_path, 'descriptive_mining', 'reglas_de_asociacion_Empleados.xlsx')
reglas.to_excel('./reglas de asociacion Empleados.xlsx')